# Demos: Lecture 4

In [1]:
import pennylane as qml
import numpy as np

/opt/conda/envs/pennylane/lib/python3.8/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


## Demo 1: `qml.ctrl`

In [7]:
def some_function():
    qml.PauliX(wires=1)
    qml.CNOT(wires=[1, 2])
    
    qml.Hadamard(wires=2)
    qml.CRX(0.3, wires=[2, 1])

In [8]:
dev = qml.device('default.qubit', wires=3)

@qml.qnode(dev)
def control_the_thing():
    qml.Hadamard(wires=0)
    
    qml.ctrl(some_function, control=0)()
    
    return qml.state()

In [9]:
control_the_thing()

tensor([ 7.07106781e-01+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        -3.85185989e-34-4.90653893e-18j,  2.08166817e-17+7.47190662e-02j,
         5.00000000e-01+0.00000000e+00j, -4.94385539e-01+8.32667268e-17j], requires_grad=True)

In [10]:
print(qml.draw(control_the_thing, expansion_strategy='device')())

 0: ──H──╭C──╭C──╭ControlledPhaseShift(1.57)──╭C─────────╭ControlledPhaseShift(1.57)──╭C─────────╭C─────────╭C──╭C──────────╭C──╭C──────────╭┤ State 
 1: ─────╰X──├C──│────────────────────────────│──────────│────────────────────────────╰RZ(1.57)──╰RY(0.15)──├X──╰RY(-0.15)──├X──╰RZ(-1.57)──├┤ State 
 2: ─────────╰X──╰ControlledPhaseShift(1.57)──╰RX(1.57)──╰ControlledPhaseShift(1.57)────────────────────────╰C──────────────╰C──────────────╰┤ State 



## Demo 2: multi-qubit measurements

In [21]:
dev = qml.device('default.qubit', wires=3)#, shots=10)

@qml.qnode(dev)
def something_parametrized(x, y):
    qml.Hadamard(wires=0)
    qml.CRX(x, wires=[0, 1])
    qml.CRY(y, wires=[1, 2])
    
    return qml.probs(wires=[0])

In [22]:
something_parametrized(0.1, 0.2)

tensor([0.5, 0.5], requires_grad=True)

## Demo 3: multi-qubit expectation values

In [23]:
dev = qml.device('default.qubit', wires=3)#, shots=10)

@qml.qnode(dev)
def something_parametrized(x, y):
    qml.Hadamard(wires=0)
    qml.CRX(x, wires=[0, 1])
    qml.CRY(y, wires=[1, 2])
    
    return qml.expval(qml.PauliZ(0)), qml.expval(qml.PauliZ(1)), qml.expval(qml.PauliZ(2))

In [24]:
something_parametrized(0.3, 0.4)

tensor([5.55111512e-17, 9.77668245e-01, 9.99118577e-01], requires_grad=True)

In [25]:
dev = qml.device('default.qubit', wires=3)#, shots=10)

@qml.qnode(dev)
def something_parametrized(x, y):
    qml.Hadamard(wires=0)
    qml.CRX(x, wires=[0, 1])
    qml.CRY(y, wires=[1, 2])
    
    return qml.expval(qml.PauliX(0) @ qml.PauliX(1) @ qml.PauliY(2))

In [26]:
something_parametrized(0.3, 0.4)

tensor(-0.02968877, requires_grad=True)

## Demo 4: superdense coding

<img src="fig/superdense.png" width="600px">

In [29]:
dev = qml.device('default.qubit', wires=2, shots=1)

def create_entangled_state(wires=None):
    qml.Hadamard(wires=wires[0])
    qml.CNOT(wires=[wires[0], wires[1]])
    
@qml.qnode(dev)
def superdense_coding(b1=0, b2=0):
    create_entangled_state(wires=[0, 1])
    
    if b1 == 1:
        qml.PauliZ(wires=0)
        
    if b2 == 1:
        qml.PauliX(wires=0)
        
    qml.adjoint(create_entangled_state)(wires=[0, 1])
    
    return qml.sample()

In [33]:
superdense_coding(b1=1, b2=0)

tensor([1, 0], requires_grad=True)

## Demo 5: teleportation 

<img src="fig/teleportation_deferred.png" width="800px">